<div align="center">
  <img src="../assets/images/hackathon.png" alt="Holistic AI Hackathon Logo" width="600"/>
</div>

**Event**: [hackathon.holisticai.com](https://hackathon.holisticai.com)

---


# Tutorial 5: Observability

**Master deep observability and debugging for production agents**

Learn to enable LangSmith tracing, view detailed execution traces, add metadata for filtering, collect user feedback, and query runs programmatically.

## What You'll Learn

1. Enable LangSmith tracing
2. View detailed execution traces
3. Add metadata for filtering
4. Collect user feedback
5. Query runs programmatically
6. Monitor complex agents with tools

## Why Observe?

- **Debugging** - Understand what your agent is doing
- **Quality assurance** - Track user satisfaction
- **Performance** - Identify bottlenecks
- **Transparency** - See agent decision-making process
- **Production monitoring** - Catch issues before users do
- **Performance optimization** - Find and fix bottlenecks

---

## Prerequisites

- Basic Python knowledge
- Recommended: Completed tutorials 01-03
- Time: ~20 minutes
- **Holistic AI Bedrock API** (recommended) - Credentials will be provided during the hackathon event
- **OpenAI API key** (optional alternative) - Get at https://platform.openai.com/api-keys
- **LangSmith API key** (required for this tutorial) - Get at https://smith.langchain.com (free)

**API Guide**: [../assets/api-guide.pdf](../assets/api-guide.pdf)

**Note:** This tutorial is completely self-contained and uses only official packages!


## Step 0: Install Dependencies

Run this cell to install all required packages for LangSmith observability.



In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load from .env file in parent directory
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    print("📄 Loaded configuration from .env file")
else:
    print("⚠️  No .env file found - using environment variables")

# Verify API keys
print("\n🔑 API Key Status:")
if os.getenv('HOLISTIC_AI_TEAM_ID') and os.getenv('HOLISTIC_AI_API_TOKEN'):
    print("  ✅ Holistic AI Bedrock credentials loaded")
elif os.getenv('OPENAI_API_KEY'):
    print("  ⚠️  OpenAI API key loaded")
else:
    print("  ⚠️  No API keys found")

if os.getenv('LANGSMITH_API_KEY'):
    print("  ✅ LangSmith API key loaded")
    os.environ["LANGSMITH_TRACING"] = "true"
    os.environ["LANGSMITH_PROJECT"] = "hackathon-2025"
else:
    print("  ⚠️  LangSmith API key not found - tracing disabled")

print("\n📁 Working directory:", Path.cwd())

# Import Holistic AI Bedrock helper
import sys
try:
    sys.path.insert(0, '../core')
    from react_agent.holistic_ai_bedrock import get_chat_model
    print("\n✅ Holistic AI Bedrock helper loaded")
except ImportError:
    print("\n⚠️  Could not import from core - will use OpenAI only")

# Import official packages
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

print("\n✅ All imports successful!")


## Step 1: Environment Setup

You'll need API keys for this tutorial:

### Recommended: Holistic AI Bedrock API
Credentials will be provided during the hackathon event.

### Optional: OpenAI API Key
Get your key at: https://platform.openai.com/api-keys (alternative)

### Required: LangSmith API Key
1. Sign up for free at: https://smith.langchain.com
2. Go to Settings > API Keys
3. Create a new API key

**Note:** This tutorial requires LangSmith for full functionality. Holistic AI Bedrock is recommended, OpenAI is optional.

### Configure Your Keys

**Option 1: Direct Setup (Quick Start)**
- Uncomment lines in the cell below and add your keys

**Option 2: Environment File (Recommended)**
- Create a `.env` file in the parent directory with:
```
HOLISTIC_AI_TEAM_ID=your-team-id-here
HOLISTIC_AI_API_TOKEN=your-api-token-here
OPENAI_API_KEY=your-openai-key-here  # Optional
LANGSMITH_API_KEY=your-langsmith-key-here
LANGSMITH_PROJECT=hackathon-2026
```

In [37]:
import os
import time
import uuid
from dotenv import load_dotenv

# ============================================
# OPTION 1: Set API keys directly (Quick Start)
# ============================================
# Uncomment and set your keys here:
# Recommended: Holistic AI Bedrock
# os.environ["HOLISTIC_AI_TEAM_ID"] = "your-team-id-here"
# os.environ["HOLISTIC_AI_API_TOKEN"] = "your-api-token-here"
# Optional: OpenAI
# os.environ["OPENAI_API_KEY"] = "your-openai-key-here"
# os.environ["LANGSMITH_API_KEY"] = "your-langsmith-key-here"  # Required for LangSmith tracing
# os.environ["LANGSMITH_PROJECT"] = "hackathon-2026"  # Optional
# os.environ["LANGSMITH_TRACING"] = "true"  # Required for LangGraph tracing
# os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"  # Optional (default)

# ============================================
# OPTION 2: Load from .env file (Recommended)
# ============================================
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    print("Loaded configuration from .env file")
else:
    print("WARNING: No .env file found - using environment variables or hardcoded keys")

# Import official packages
# Import Holistic AI Bedrock helper function
# Import from core module
try:
    import sys
    sys.path.insert(0, '../core')
    from react_agent.holistic_ai_bedrock import HolisticAIBedrockChat, get_chat_model
    print("✅ Holistic AI Bedrock helper function loaded")
except ImportError:
    print("⚠️  Could not import from core")

from langgraph.prebuilt import create_react_agent
# Optional: OpenAI (if not using Bedrock)
# from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

# Import monitoring tools
import tiktoken
from codecarbon import EmissionsTracker
from langsmith import Client

# Check API keys (Holistic AI Bedrock recommended, OpenAI optional)
# Check Holistic AI Bedrock (recommended)
if os.getenv('HOLISTIC_AI_TEAM_ID') and os.getenv('HOLISTIC_AI_API_TOKEN'):
    print("✅ Holistic AI Bedrock credentials loaded (will use Bedrock)")

# Check OpenAI (optional)
if os.getenv('OPENAI_API_KEY'):
    key_preview = os.getenv('OPENAI_API_KEY')[:10] + "..."
    print(f"OpenAI API key loaded: {key_preview}")
else:
    print("ℹ️  OpenAI API key not found - optional, will use Bedrock if available")

# Check LangSmith API key (required for this tutorial)
if os.getenv('LANGSMITH_API_KEY'):
    ls_key = os.getenv('LANGSMITH_API_KEY')[:10] + "..."
    print(f"LangSmith API key loaded: {ls_key}")
    
    # Set required LangGraph tracing environment variables if not already set
    if not os.getenv('LANGSMITH_TRACING'):
        os.environ['LANGSMITH_TRACING'] = 'true'
        print("  LANGSMITH_TRACING set to 'true' (required for LangGraph tracing)")
    
    if not os.getenv('LANGSMITH_ENDPOINT'):
        os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
        print("  LANGSMITH_ENDPOINT set to 'https://api.smith.langchain.com'")
    
    langsmith_project = os.getenv('LANGSMITH_PROJECT', 'default')
    print(f"  LangSmith project: {langsmith_project}")
    print("  LangSmith tracing will be fully functional!")
else:
    print("ERROR: LangSmith API key not found - tracing will not work!")
    print("  Get a free key at: https://smith.langchain.com")
    print("  This tutorial requires LangSmith to function properly!")

print("\nAll imports successful!")


Loaded configuration from .env file
✅ Holistic AI Bedrock helper function loaded
✅ Holistic AI Bedrock credentials loaded (will use Bedrock)
OpenAI API key loaded: sk-proj-10...
LangSmith API key loaded: lsv2_pt_10...
  LangSmith project: pr-loyal-quota-66
  LangSmith tracing will be fully functional!

All imports successful!


---

# Observability

Deep tracing and debugging for production agents.

**Note:** This section requires a LangSmith API key. If you don't have one, you can still read through to understand the concepts.

## Step 2: Enable LangSmith Tracing

LangGraph agents automatically trace to LangSmith when the required environment variables are set:
- `LANGSMITH_API_KEY` - Your LangSmith API key
- `LANGSMITH_TRACING=true` - Required to enable automatic tracing
- `LANGSMITH_ENDPOINT` - LangSmith API endpoint (default: https://api.smith.langchain.com)
- `LANGSMITH_PROJECT` - Project name for organizing traces (optional)

**Note:** These are automatically set in Step 1 when `LANGSMITH_API_KEY` is detected. No additional configuration needed!


Every time you call `agent.invoke()`, a trace is automatically sent to LangSmith with:
- Full conversation history
- Token usage breakdown
- Latency at each step
- Model parameters used
- Tool calls and outputs (if using tools)

In [38]:
# Create agent with automatic LangSmith tracing
# Use get_chat_model() - uses Holistic AI Bedrock by default (recommended)
llm_traced = get_chat_model("claude-3-5-sonnet")  # Uses Holistic AI Bedrock (recommended)
agent_traced = create_react_agent(llm_traced, tools=[])

langsmith_project = os.getenv('LANGSMITH_PROJECT', 'default')

print("Agent created with automatic LangSmith tracing!")
print(f"  Model: claude-3-5-sonnet (via Bedrock if available)")
print(f"  Project: {langsmith_project}")
print(f"\nEvery agent.invoke() call will be traced to:")
print(f"  https://smith.langchain.com")

if not os.getenv('LANGSMITH_API_KEY'):
    print("\nWARNING: No LangSmith API key found!")
    print("  Tracing will not work without it.")
    print("  Get a free key at: https://smith.langchain.com")

Agent created with automatic LangSmith tracing!
  Model: claude-3-5-sonnet (via Bedrock if available)
  Project: pr-loyal-quota-66

Every agent.invoke() call will be traced to:
  https://smith.langchain.com


## Step 3: View Traces in LangSmith UI

Let's run a query and view the trace in LangSmith.

The trace will show you:
- Input messages
- Model responses
- Token counts (prompt and completion)
- Latency for each step
- Total cost estimate

In [39]:
# Generate a unique run ID for tracking
run_id = str(uuid.uuid4())

print(f"Run ID: {run_id}")
print("\nRunning agent...\n")

# Run the agent with metadata
start_time = time.time()
result = agent_traced.invoke(
    {"messages": [("user", "Explain machine learning in one sentence.")]},
    {"run_id": run_id, "tags": ["tutorial", "observability"]}
)
elapsed = time.time() - start_time

response = result['messages'][-1].content

print("Response:")
print(f"  {response}")
print(f"\nLatency: {elapsed:.2f}s")
print("\n" + "="*70)
print("View this trace in LangSmith:")

# Try to get the exact URL
if os.getenv('LANGSMITH_API_KEY'):
    try:
        ls_client = Client()
        run_url = ls_client.read_run(run_id).url
        print(f"  {run_url}")
    except Exception:
        print(f"  https://smith.langchain.com")
        print(f"  Project: {langsmith_project}")
        print(f"  Search for run ID: {run_id}")
else:
    print("  (LangSmith API key not set - tracing disabled)")

print("\nThe trace shows:")
print("  - Full conversation history")
print("  - Token usage breakdown")
print("  - Latency at each step")
print("  - Model parameters used")

Run ID: b844fabd-03fc-4e92-99e0-db3815d6718b

Running agent...



Response:
  Machine learning is a branch of artificial intelligence where computer systems learn patterns from data to make predictions or decisions without being explicitly programmed.

Latency: 1.58s

View this trace in LangSmith:
  https://smith.langchain.com
  Project: pr-loyal-quota-66
  Search for run ID: b844fabd-03fc-4e92-99e0-db3815d6718b

The trace shows:
  - Full conversation history
  - Token usage breakdown
  - Latency at each step
  - Model parameters used


## Step 4: Add Metadata for Filtering

Metadata helps you filter and analyze runs in production. You can add custom metadata to track:
- User IDs
- Session IDs
- Feature flags
- A/B test variants
- Environment (dev/staging/prod)

This makes it easy to:
- Filter runs by user
- Compare A/B test variants
- Debug user-specific issues
- Track feature adoption

In [40]:
# Run with rich metadata
run_id = str(uuid.uuid4())
metadata = {
    "user_id": "demo_user_123",
    "model": "claude-3-5-sonnet",  # Using Bedrock
    "version": "v1.0",
    "environment": "tutorial"
}

result = agent_traced.invoke(
    {"messages": [("user", "What is quantum computing?")]},
    {"run_id": run_id, "tags": ["tutorial", "metadata-demo"], "metadata": metadata}
)

print("Run completed with metadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

# Get URL from LangSmith
if os.getenv('LANGSMITH_API_KEY'):
    try:
        ls_client = Client()
        run_url = ls_client.read_run(run_id).url
        print(f"\nView at: {run_url}")
    except Exception:
        print(f"\nView in LangSmith:")
        print(f"  https://smith.langchain.com")
        print(f"  Project: {langsmith_project}")
        print(f"  Run ID: {run_id}")
else:
    print("\n(LangSmith API key not set - tracing disabled)")

Run completed with metadata:
  user_id: demo_user_123
  model: claude-3-5-sonnet
  version: v1.0
  environment: tutorial

View at: https://smith.langchain.com/o/1961c9a6-cf5c-4882-86b8-9028e1683672/projects/p/c17bf9b5-dbc0-4a35-8f65-5adc9a5b2911/r/d99078fc-ca14-478c-8053-bed4b2082612?trace_id=d99078fc-ca14-478c-8053-bed4b2082612&start_time=2025-11-11T00:37:04.628442


## Step 5: Collect User Feedback

User feedback is crucial for improving your agents. LangSmith makes it easy to:
- Collect thumbs up/down feedback
- Add user comments
- Track satisfaction over time
- Find problematic runs

In production, you would collect this feedback from your UI and send it to LangSmith.

In [41]:
if os.getenv('LANGSMITH_API_KEY'):
    try:
        ls_client = Client()
        
        # Simulate user feedback (in production, this would come from UI)
        user_score = 1.0  # 1.0 = thumbs up, 0.0 = thumbs down
        user_comment = "Great explanation!"
        
        # Log feedback
        ls_client.create_feedback(
            run_id,
            key="user-score",
            score=user_score,
            comment=user_comment
        )
        
        print("Feedback logged successfully!")
        print(f"  Score: {'thumbs up' if user_score > 0.5 else 'thumbs down'}")
        print(f"  Comment: {user_comment}")
        print("\nIn LangSmith, you can:")
        print("  - Filter runs by feedback score")
        print("  - Identify problematic runs")
        print("  - Track user satisfaction trends")
    except Exception as e:
        print(f"Could not log feedback: {e}")
        print("Make sure you have a valid LangSmith API key")
else:
    print("LangSmith API key not set - feedback logging disabled")
    print("\nIn production, user feedback helps you:")
    print("  - Identify problematic responses")
    print("  - Track satisfaction over time")
    print("  - Find runs that need investigation")

Feedback logged successfully!
  Score: thumbs up
  Comment: Great explanation!

In LangSmith, you can:
  - Filter runs by feedback score
  - Identify problematic runs
  - Track user satisfaction trends


## Step 6: Query Runs Programmatically

Use the LangSmith SDK to query and analyze runs. This is useful for:
- Building custom dashboards
- Automated monitoring
- Data analysis
- Quality assurance

You can filter runs by:
- Tags, metadata, feedback
- Time range, latency, tokens
- Error status, model used

In [42]:
from datetime import datetime, timedelta

if os.getenv('LANGSMITH_API_KEY'):
    try:
        ls_client = Client()
        
        # Query recent runs from our project
        recent_runs = ls_client.list_runs(
            project_name=langsmith_project,
            start_time=datetime.now() - timedelta(hours=1),
            limit=5
        )
        
        print("Recent Runs (last hour):")
        print("="*70)
        for i, run in enumerate(recent_runs, 1):
            print(f"\n{i}. Run ID: {run.id}")
            print(f"   Status: {run.status}")
            print(f"   Latency: {run.latency:.2f}s" if run.latency else "   Latency: N/A")
            if run.feedback_stats:
                print(f"   Feedback: {run.feedback_stats}")
            if i >= 5:
                break
        
        print("\nYou can query by:")
        print("  - Tags, metadata, feedback")
        print("  - Time range, latency, tokens")
        print("  - Error status, model used")
    except Exception as e:
        print(f"Could not query runs: {e}")
        print("Make sure you have a valid LangSmith API key")
else:
    print("LangSmith API key not set - querying disabled")
    print("\nIn production, programmatic querying enables:")
    print("  - Custom dashboards")
    print("  - Automated alerts")
    print("  - Quality assurance checks")
    print("  - Performance analysis")

Recent Runs (last hour):

1. Run ID: 38a302e1-f11c-4c67-aac2-601e6645c214
   Status: success
   Latency: 6.92s

2. Run ID: 2bfac2d1-ddf0-4697-a1b7-9e895afc1e48
   Status: success
   Latency: 0.00s

3. Run ID: 15c7dc8b-b16b-4099-9f14-239fd40d0839
   Status: success
   Latency: 6.92s

4. Run ID: 554105e7-bf59-4147-a7c8-f30f9500eadf
   Status: success
   Latency: 6.92s

5. Run ID: edcd6fdd-ba92-4e68-a824-20173f72aa48
   Status: success
   Latency: 6.92s

You can query by:
  - Tags, metadata, feedback
  - Time range, latency, tokens
  - Error status, model used


## Step 7: Monitor Complex Agent with Tools

Let's trace a more complex agent that uses tools. This shows how LangSmith helps you understand:
- Agent reasoning steps
- Tool selection and execution
- Tool outputs
- Final synthesis

This is especially valuable when debugging why an agent chose certain tools or produced unexpected results.

In [43]:
# Create a simple demo tool
@tool
def get_current_date(format: str = "YYYY-MM-DD") -> str:
    """Get the current date in the specified format."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

# Create agent with tool
# Use get_chat_model() - uses Holistic AI Bedrock by default (recommended)
llm_with_tool = get_chat_model("claude-3-5-sonnet")  # Uses Holistic AI Bedrock (recommended)
agent_with_tool = create_react_agent(llm_with_tool, tools=[get_current_date])

run_id = str(uuid.uuid4())
print(f"Complex Run ID: {run_id}")
print("\nRunning agent with tool...\n")

result = agent_with_tool.invoke(
    {"messages": [("user", "What is today's date?")]},
    {"run_id": run_id, "tags": ["tool-use", "complex"]}
)

response = result['messages'][-1].content
print(f"Response: {response}")

print("\nComplex trace complete!")

# Get URL
if os.getenv('LANGSMITH_API_KEY'):
    try:
        ls_client = Client()
        run_url = ls_client.read_run(run_id).url
        print(f"\nView detailed trace: {run_url}")
    except Exception:
        print(f"\nFind trace in LangSmith:")
        print(f"  https://smith.langchain.com")
        print(f"  Project: {langsmith_project}")
        print(f"  Search run ID: {run_id}")
else:
    print("\n(LangSmith API key not set - tracing disabled)")

print("\nThe trace shows:")
print("  - Agent reasoning steps")
print("  - Tool calls (get_current_date)")
print("  - Tool outputs")
print("  - Final response synthesis")

Complex Run ID: 0ab39e2f-5283-4f55-a3ac-20e46671cbec

Running agent with tool...

Response: Today's date is 2025-11-11.

Complex trace complete!

Find trace in LangSmith:
  https://smith.langchain.com
  Project: pr-loyal-quota-66
  Search run ID: 0ab39e2f-5283-4f55-a3ac-20e46671cbec

The trace shows:
  - Agent reasoning steps
  - Tool calls (get_current_date)
  - Tool outputs
  - Final response synthesis


---

## Summary

Congratulations! You've mastered observability and debugging with LangSmith.

### What You Learned

1. **Enable Tracing** - Automatic tracing with LangSmith API key
2. **View Traces** - See full execution in LangSmith UI
3. **Add Metadata** - Filter by user, environment, variant
4. **Collect Feedback** - Track user satisfaction
5. **Query Runs** - Programmatic analysis with LangSmith SDK
6. **Debug Complex Agents** - Understand tool usage and reasoning

### Key Insights

- LangSmith provides automatic tracing for LangGraph agents
- Metadata enables powerful filtering and analysis
- User feedback helps identify problematic runs
- Programmatic querying enables custom dashboards
- Tracing shows full conversation history, token usage, and latency

### Production Best Practices

1. **Always add metadata** - Makes filtering and analysis easier
2. **Collect user feedback** - Essential for improvement
3. **Set up alerts** - Monitor error rates, latency spikes, costs
4. **Regular analysis** - Review traces to find optimization opportunities
5. **Use tags** - Organize runs by feature, version, environment

### What's Next?

Continue your learning journey:
- **04_model_monitoring.ipynb** - Learn performance monitoring and cost tracking
- **06_benchmark_evaluation.ipynb** - Test agents on PhD-level questions
- **Advanced**: Set up production monitoring dashboards
- **Advanced**: Create automated quality assurance pipelines

### Resources

- [LangSmith Documentation](https://docs.smith.langchain.com)
- [Observability Guide](https://docs.smith.langchain.com/observability)
- [LangGraph Tracing](https://langchain-ai.github.io/langgraph/how-tos/tracing/)
